# RKD HTTP JSON with Python Example: Authentication

## Overview
The [Refinitiv Knowledge Direct (RKD) API](https://developers.refinitiv.com/en/api-catalog/refinitiv-knowledge-direct/refinitiv-knowledge-direct-api-rkd-api) (formerly known as TRKD API)integrates into your website, trading platform, company intranet/extranet, advisory portal and mobile applications to provide up-to-date financial market data, news and analytics and powerful investment tools.

RKD offers a wide range of Refinitiv' information and services delivered in a request-response scenario via web services using today's industry standard protocols (SOAP/XML and REST/JSON). Connectivity can be via HTTP and HTTPS, over the Internet or Delivery Direct. All data are snapshot (non-streaming) data.

This is an example project that shows how to implement RKD HTTP JSON client with Python programming lanugage in Jupyter Notebook.

### RKD JSON application implementation process
The JSON application requires the following steps to consume data from RKD API services
1. Authentication with RKD Authentication service to get an authen token
2. Send a request message with the required input information and authen token to the interested RKD service

In [1]:
# import requires libraries for HTTP Request and JSON Operation

import os
import requests
import json
from dotenv import load_dotenv

In [2]:
# HTTP operations function
def doSendRequest(url, requestMsg, headers):
    result = None
    try:
        result = requests.post(url, data=json.dumps(requestMsg), headers=headers)
    except requests.exceptions.RequestException as e:
        print('Exception!!!')
        print(e)
    return result

## main process

You should save a text file with **filename** `.env` or Environment Variables having the following configurations:

```
#RKD Access Credentials
RKD_USERNAME=<RKD Username>
RKD_PASSWORD=<RKD Password>
RKD_APP_ID=<RKD App ID>
```

In [ ]:
%load_ext dotenv
%dotenv

# Input your RKD credentials here

username = os.getenv('RKD_USERNAME')
password = os.getenv('RKD_PASSWORD')
appid = os.getenv('RKD_APP_ID')

In [5]:
# Authentication parameters

token = None
expire = None

#### RKD Service Token Detail
##### RKD Service Token URL and Header

The URL enponint for the RKD Service Token is following:
[https://api.rkd.refinitiv.com/api/TokenManagement/TokenManagement.svc/REST/Anonymous/TokenManagement_1/CreateServiceToken_1](https://api.rkd.refinitiv.com/api/TokenManagement/TokenManagement.svc/REST/Anonymous/TokenManagement_1/CreateServiceToken_1)

Header: 
- Content-type = application/json;charset=utf-8
Method:
- Post

##### RKD Service TOKEN Request Message

The CreateServiceToken_1 operation requires the following information to perform authentication
- ApplicationID
- Username
- Password

The request message structure is following
```
{
    “CreateServiceToken_Request_1”:{
      “ApplicationID”: <application id>,
      “Username”: <username>,
      “Password”: <password>
   }
}

```
The example of the response message is shown below
```
{
  "CreateServiceToken_Response_1": {
    "Expiration": "2016-09-26T09:42:54.4335265Z",
    "Token": "674E12E4EF35F181602672D5529D98379D4B42216057C7FF…"
  }
}
```
- Token: an encrypted, expiring string that securely identifies the service user  (aka service Token)
- Expiration: Token expires after a configurable time period. The default expiration is 90 minutes.

In [6]:
# RKD Authentication Post Request message

authenMsg = {'CreateServiceToken_Request_1': {'ApplicationID': appid, 'Username': username, 'Password': password}}

In [7]:
# RKD Authentication Service URL

authenURL = (
    'https://api.rkd.refinitiv.com/api/'
    'TokenManagement/TokenManagement.svc/REST/'
    'Anonymous/TokenManagement_1/CreateServiceToken_1'
)

In [8]:
# RKD Authentication Request headers

authen_headers = {'content-type': 'application/json;charset=utf-8'}

In [9]:
print('############### Sending Authentication request message to RKD ###############')

############### Sending Authentication request message to RKD ###############


In [10]:
# Send Authentication request

authenResult = doSendRequest(authenURL, authenMsg, authen_headers)

In [ ]:
if authenResult and authenResult.status_code == 200:
    print('Authenticaion success')
    print('JSON resonse: %s' % (json.dumps(authenResult.json(),sort_keys=True, indent=2, separators=(',', ':'))))
    token = authenResult.json()['CreateServiceToken_Response_1']['Token']
    expire = authenResult.json()['CreateServiceToken_Response_1']['Expiration']
elif authenResult.status_code != 200:
    print('Authenticaion fail with status code %s' % authenResult.status_code)
    if authenResult.status_code == 500:
        print('Error: %s' % (json.dumps(authenResult.json(), sort_keys=True, indent=2, separators=(',', ':'))))

In [ ]:
# Print Authentication information

print('Token = : %s' % token)
print('Expiration = %s' % expire)

## Conclusion
All RKD HTTP JSON applications require authenticaiton to access RKD data. The application needs to request for Service Token from RKD server, then keeps a response Service Token for later use in other request message header.

## References
For further details, please check out the following resources:
* [Refinitiv Knowledge Direct API page](https://developers.refinitiv.com/en/api-catalog/refinitiv-knowledge-direct/refinitiv-knowledge-direct-api-rkd-api) on the [Refinitiv Developer Community](https://developers.refinitiv.com) website.
* [Refinitiv Knowledge Direct API Catalog](https://support-portal.rkd.refinitiv.com/SupportSite/Home/Index) website.
* RKD Article: [How to implement RKD JSON application with Python chapter 1: the basic](https://developers.refinitiv.com/article/how-implement-trkd-json-application-python-chapter-1-basic)
* [RKD Tutorial](https://developers.refinitiv.com/en/api-catalog/refinitiv-knowledge-direct/refinitiv-knowledge-direct-api-rkd-api/tutorials).

For any question related to this tutorial or RKD API, please use the Developer Community [Q&A Forum](https://community.developers.refinitiv.com/spaces/51/view.html).